<a href="https://colab.research.google.com/github/TPincher/IntroToPyTorch/blob/main/ConvolutionalNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Convert MNIST Image Files into a Tensor of 4-dimensions (# of images, height, width, colour channels)
transform = transforms.ToTensor()

In [4]:
# Train Data
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 64706256.90it/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 17671144.25it/s]

Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 15926683.13it/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17162638.53it/s]


Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [5]:
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [6]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [23]:
# Create a small batch size for images. Usually 2 to 4, but will use 10 for this.
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [24]:
# Define our CNN Model
# Describe convolutional layer and what it's doing (2 convolutional layers)
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [25]:
# Grab 1 MNIST image/record
for i, (X_train, y_train) in enumerate(train_data):
  break

In [29]:
x = X_train.view(1, 1, 28, 28)

In [30]:
# Perform our first convolution
  #Rectified Linear Unit for our activation function
x = F.relu(conv1(x))

In [32]:
# 1 single image, 6 is the filters we asked for, 26x26 is now the size of the image. It's come down from 28x28 as the outer edge of pixels is dropped unless padding is handled
x.shape

torch.Size([1, 6, 26, 26])

In [33]:
# pass through the pooling layer
  # Kernel of 2 and stride of 2
x = F.max_pool2d(x, 2, 2)

In [34]:
# we get 13x13 here as each block of 2x2 has been shrunk down into a single point
x.shape

torch.Size([1, 6, 13, 13])

In [36]:
# Do our second convolutional layer
x = F.relu(conv2(x))

In [37]:
# Again, no padding so outermost layer of pixels is lost
x.shape

torch.Size([1, 16, 11, 11])

In [38]:
# Pooling layer
x = F.max_pool2d(x, 2, 2)

In [39]:
# 11 rounds down to 5 because data can't be invented to round up.
x.shape

torch.Size([1, 16, 5, 5])

In [45]:
# Model Class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(1, 16, 3, 1)
    # Fully connected layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    # 2x2 kernel and a stride of 2
    X = F.max_pool2d(X, 2, 2)
    X = F.relu(self.conv2(X))
    # 2x2 kernel and a stride of 2
    X = F.max_pool2d(X, 2, 2)

    # Re-View to flatten it out
    X = X.view(-1, 16*5*5)

    # Fully connected layers
    X = F.relu(self.fc1(X))
    X = F.reul(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1)

In [46]:
# Create an instance of the Model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [47]:
# Loss function optimizer
criterion = nn.CrossEntropyLoss()
# The smaller the learning rate, the long it will take to train
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [50]:
import time
start_time = time.time()

# Create variables to track things so we can break this apart after the fact
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For loop for epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0

  # Train
  for b,(X_train, y_train) in enumerate(train_loader):
    # Start batches at 1
    b+=1
    # Get predicted values from the training set. These are 2d, not flattened.
    y_pred = model(X_train)
    # Compare predictions to correct answers in y_train to see how off we are.
    loss = criterion(y_pred, y_train)
    # Add up the number of correct predictions, indexed off the first point.
    predicted = torch.max(y_pred.data, 1)[1]
    # How many we got correct from this specific batch. True = 1, False = 0.
    batch_corr = (predicted == y_train).sum()
    # Keep track as we go along in training
    trn_corr += batch_corr

    # Update parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print out results
    if b%600 == 0:
      print(f'Epoch: {i} Batch: {b} Loss: {loss.item()}')

  train_losses.append(loss)
  train_correct.append(trn.corr)

  # Test
  # No gradient so we don't update our weights and biases with test data
  with torch.no_grad():
    for b,(X_test, y_test) in enumerate(test_loader):
      y_val = model(X_test)
      # Addinmg up correct predictions
      predicted = torch.max(y_val.data, 1)[1]
      tst_core += (predicted == y_test).sum()

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)


current_time = time.time()
total = current_time - start_time
print(f'Training took: {total/60} minutes')

RuntimeError: Given groups=1, weight of size [16, 1, 3, 3], expected input[10, 6, 13, 13] to have 1 channels, but got 6 channels instead